In [1]:
import pandas as pd
import psycopg2
import json

In [2]:
business_raw = pd.read_csv('csv_files/yelp_academic_dataset_business.csv')
business_raw.index +=1

In [3]:
business_raw.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
1,2818 E Camino Acequia Drive,{},1SWheh84yJXfytovILXOAQ,"Golf, Active Life",Phoenix,NaN,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ
2,30 Eglinton Avenue W,"{'GoodForMeal': ""{'dessert': False, 'latenight...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
3,"1210 8th Street SW, Unit 220","{'BusinessParking': ""{'garage': False, 'street...",fcXOEZdXYeZqnQ3lGlOXmg,"Local Services, Professional Services, Compute...",Calgary,"{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ...",1,51.041771,-114.081109,Nucleus Information Service,T2R 1L3,5,2.0,AB
4,"6955 N Durango Dr, Ste 1116","{'BusinessParking': ""{'garage': False, 'street...",f2ZWZPENViL92BrFsIgR6w,"Beauty & Spas, Hair Salons",Las Vegas,"{'Monday': '8:30-20:0', 'Tuesday': '8:30-20:0'...",1,36.287312,-115.289540,Amazing Cuts,89149,116,4.5,NV
5,4 E University Dr,{},6KgGE8B1RsR7jc9R5nuH0Q,"American (Traditional), Restaurants",Tempe,NaN,0,33.422192,-111.939615,Ruby Tuesday,85281,9,2.5,AZ


In [74]:
eval(business_raw["attributes"][2])

{'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}",
 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",
 'NoiseLevel': "u'loud'",
 'Ambience': "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}"}

# Parse businesses ids

In [4]:
business_ids = business_raw["business_id"].reset_index().set_index("business_id")["index"].to_dict()
del business_raw["business_id"]
business_raw.head()

,address,attributes,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
1,2818 E Camino Acequia Drive,{},"Golf, Active Life",Phoenix,NaN,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ
2,30 Eglinton Avenue W,"{'GoodForMeal': ""{'dessert': False, 'latenight...","Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
3,"1210 8th Street SW, Unit 220","{'BusinessParking': ""{'garage': False, 'street...","Local Services, Professional Services, Compute...",Calgary,"{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ...",1,51.041771,-114.081109,Nucleus Information Service,T2R 1L3,5,2.0,AB
4,"6955 N Durango Dr, Ste 1116","{'BusinessParking': ""{'garage': False, 'street...","Beauty & Spas, Hair Salons",Las Vegas,"{'Monday': '8:30-20:0', 'Tuesday': '8:30-20:0'...",1,36.287312,-115.289540,Amazing Cuts,89149,116,4.5,NV
5,4 E University Dr,{},"American (Traditional), Restaurants",Tempe,NaN,0,33.422192,-111.939615,Ruby Tuesday,85281,9,2.5,AZ


# Parse states

In [76]:
state_table = pd.Series(business_raw["state"].unique(), name="state")
state_table.index += 1
state_dict = state_table.reset_index().set_index("state")["index"].to_dict()
business_raw["state"] = business_raw["state"].apply(lambda s: state_dict[s])
business_raw.head()

,address,attributes,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
1,2818 E Camino Acequia Drive,{},"Golf, Active Life",Phoenix,NaN,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,1
2,30 Eglinton Avenue W,"{'GoodForMeal': ""{'dessert': False, 'latenight...","Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,2
3,"1210 8th Street SW, Unit 220","{'BusinessParking': ""{'garage': False, 'street...","Local Services, Professional Services, Compute...",Calgary,"{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ...",1,51.041771,-114.081109,Nucleus Information Service,T2R 1L3,5,2.0,3
4,"6955 N Durango Dr, Ste 1116","{'BusinessParking': ""{'garage': False, 'street...","Beauty & Spas, Hair Salons",Las Vegas,"{'Monday': '8:30-20:0', 'Tuesday': '8:30-20:0'...",1,36.287312,-115.289540,Amazing Cuts,89149,116,4.5,4
5,4 E University Dr,{},"American (Traditional), Restaurants",Tempe,NaN,0,33.422192,-111.939615,Ruby Tuesday,85281,9,2.5,1


# Parse cities

In [77]:
city_table = pd.Series(business_raw["city"].unique(), name="city")
city_table.index += 1
city_dict = city_table.reset_index().set_index("city")["index"].to_dict()
city_state_table = business_raw[["state", "city"]].apply(lambda row: (city_dict[row["city"]], row["state"]), axis=1).unique()
del business_raw["state"]
business_raw["city"] = business_raw["city"].apply(lambda city: city_dict[city])
business_raw.head()

,address,attributes,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars
1,2818 E Camino Acequia Drive,{},"Golf, Active Life",1,NaN,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0
2,30 Eglinton Avenue W,"{'GoodForMeal': ""{'dessert': False, 'latenight...","Specialty Food, Restaurants, Dim Sum, Imported...",2,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5
3,"1210 8th Street SW, Unit 220","{'BusinessParking': ""{'garage': False, 'street...","Local Services, Professional Services, Compute...",3,"{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ...",1,51.041771,-114.081109,Nucleus Information Service,T2R 1L3,5,2.0
4,"6955 N Durango Dr, Ste 1116","{'BusinessParking': ""{'garage': False, 'street...","Beauty & Spas, Hair Salons",4,"{'Monday': '8:30-20:0', 'Tuesday': '8:30-20:0'...",1,36.287312,-115.289540,Amazing Cuts,89149,116,4.5
5,4 E University Dr,{},"American (Traditional), Restaurants",5,NaN,0,33.422192,-111.939615,Ruby Tuesday,85281,9,2.5


# Store in data base

In [ ]:
con = psycopg2.connect(database="introdb")
print("Connection opened")
cur = con.cursor()

# stores states
for name, id in state_dict.items():
    cur.execute("insert into state (id, name) values ({id}, '{name}')".format(id=id, name=name))
con.commit()

cur.execute("select * from state")
print(cur.fetchall())

con.close()
print("Connection closed")

In [ ]:
con = psycopg2.connect(database="introdb")
print("Connection opened")
cur = con.cursor()

cur.execute("select * from state")
print(cur.fetchall())

con.close()
print("Connection closed")